In [ ]:
import torch
import torchvision
from torchvision import models, transforms, datasets
import torch.nn as nn
import torch.nn.functional as func
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomAffine(0, shear=2, scale=(0.95, 1.05)),
        transforms.RandomResizedCrop(224),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test' : transforms.Compose([
        transforms.Resize(255),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])     
    ])
}

trainset = datasets.ImageFolder(root=".\\train\\",transform=data_transforms["train"])
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

testset = datasets.ImageFolder(root=".\\test\\", transform=data_transforms["test"])
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=True)

In [ ]:
images, img_class = next(iter(trainloader))

imgs = torchvision.utils.make_grid(images).numpy().transpose((1, 2, 0))
imgs *= np.array([0.229, 0.224, 0.225])
imgs += np.array([0.485, 0.456, 0.406])
imgs = np.clip(imgs, 0, 1)
plt.imshow(imgs)

In [ ]:
class Network_a(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2048, 256)
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc4 = nn.Linear(16, 4)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.dropout(func.relu(self.fc1(x)))
        x = self.dropout(func.relu(self.fc2(x)))
        x = self.dropout(func.relu(self.fc3(x)))
        x = func.log_softmax(self.fc4(x), dim=1)
        return x

class Network_b(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 64)
        self.fc4 = nn.Linear(64, 64)
        self.fc5 = nn.Linear(64, 16)
        self.fc6 = nn.Linear(16, 16)
        self.fc7 = nn.Linear(16, 4)
        self.fc8 = nn.Linear(4, 4)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.dropout(func.relu(self.fc1(x)))
        x = self.dropout(func.relu(self.fc2(x)))
        x = self.dropout(func.relu(self.fc3(x)))
        x = self.dropout(func.relu(self.fc4(x)))
        x = self.dropout(func.relu(self.fc5(x)))
        x = self.dropout(func.relu(self.fc6(x)))
        x = self.dropout(func.relu(self.fc7(x)))
        x = func.log_softmax(self.fc8(x), dim=1)
        return x

class Network_c(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 64)
        self.fc4 = nn.Linear(64, 16)
        self.fc5 = nn.Linear(16, 16)
        self.fc6 = nn.Linear(16, 4)
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.dropout(func.relu(self.fc1(x)))
        x = self.dropout(func.relu(self.fc2(x)))
        x = self.dropout(func.relu(self.fc3(x)))
        x = self.dropout(func.relu(self.fc4(x)))
        x = self.dropout(func.relu(self.fc5(x)))
        x = func.log_softmax(self.fc6(x), dim=1)
        return x

networks = [Network_a(), Network_b(), Network_c()]

In [ ]:
epochs = 4

for network in networks:
    model = models.resnet50(pretrained=True)

    for parameter in model.parameters():
        parameter.requires_grad = False

    model.fc = network
    model.to(device)
    model

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                output = model(images)
                loss = criterion(output, labels)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()*images.size(0)

        model.eval()
        acc = 0
        eval_loss = 0
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with torch.no_grad():
                output = model(images)
                loss = criterion(output, labels)
                eval_loss += loss.item()*images.size(0)

                pred_val, pred_class = torch.exp(output).topk(1, dim=1)
                is_equal = pred_class == labels.view(*pred_class.shape)
                acc += torch.mean(is_equal.type(torch.FloatTensor)).item()
        
        acc/=len(testloader)
        eval_loss/= len(testloader)
        train_loss/= len(trainloader)
        print('Epoch: {} Train Loss: {:.2f} Accuracy: {:.2f} Test Loss: {:.2f} \n'.format(epoch, train_loss, acc, eval_loss))
